# 최종 전처리 코드
##### NEWS_DIRECTORY, 즉 경로만 수정하면 문제 없이 user filtering 이전/이후(raw/not raw) 파일 생성

In [2]:
# import necessary modules
import os
import json
import time
from datetime import datetime
from random import sample
# import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import Counter
import itertools
import sys


NEWS_DIRECTORY = './Adressa_4w'
dir_list = ['history','train','test']

In [6]:
# 1. user filtering을 위해 body를 생성하는 코드


def parse_time(timestamp):
    return datetime.fromtimestamp(float(timestamp)).strftime('%Y-%m-%d %H:%M:%S')

def str_to_timestamp(string):
    return datetime.timestamp(datetime.strptime(string,'%Y-%m-%d %H:%M:%S'))


""" Step 1. make_user_news_data 정의
"""

### def make_user_news_data():
print()
print('Make_user_news_data',end='    ')
print('all_start_time:',time.strftime('%y-%m-%d %H:%M:%S'))
print()
print()
start = time.time()
first_execution = True
def make_user_news_dic(body):
    global user_dic
    global user_cnt
    global news_cnt
    global news_dic
    global index
    global behaviors_list
    global filter1_cnt
    global behaviors_user_filter_cnt
    global time_error_cnt
    global first_execution
    # 내가 추가한 것
    global time_limit_list
    

    ### 유효한 뉴스 데이터만 취급. 이로써, 뉴스 데이터는 시작부터 필터링 됨
    filter_list = ["time", "publishtime",
                "title", "userId","url", "canonicalUrl"]
    if all(filter in body.keys() for filter in filter_list):
        ### 처음 실행 때는 user_filter가 정의되지 않아 이를 제하고 실행하게 함
        if first_execution:
            pass
        else:
            if user_filter_dic.get(body['userId'],False):
                behaviors_user_filter_cnt += 1
                return
        
        ### news_dic 만들기
        body['publishtime'] = body['publishtime'].split('.')[0].replace('T',' ')
        body['time'] = parse_time(body['time'])
        
        ### 시간 역행 필터링
        if str_to_timestamp(body['time'])-str_to_timestamp(body['publishtime']) < 0:
            time_error_cnt += 1
            return
        
        ### body에 뉴스 데이터 추가
            ### 1) Url이 존재하지 않는 경우
        if body['canonicalUrl'] not in news_dic.keys():
            news_id = 'N'+str(news_cnt)
            news_dic[body['canonicalUrl']] = [news_id,body['publishtime'],body['title']]
            news_dic[body['canonicalUrl']].append([body['time']])
            news_cnt += 1
            # news_df = news_df.append({'news_id':news_id,'url':body['canonicalUrl'],'publish_time':body['publishtime'],'clicked_times':body['time'],'title':body['title']},ignore_index=True)
            if 'category1' in body.keys():
                news_dic[body['canonicalUrl']].append(body['category1'])
            else:
                news_dic[body['canonicalUrl']].append("")
            # if news_cnt == 10:
            #     break
        
            ### 2) 이미 Url이 존재하는 경우 
        else:
            news_id = news_dic[body['canonicalUrl']][0]
            news_dic[body['canonicalUrl']][3].append(body['time'])
            # target_news_index = int(news_dic[body['canonicalUrl']][1:]) - 1
            # news_id = news_df.loc[target_news_index]['news_id']
            # total_click = news_df.loc[target_news_index]['clicked_times']+',' + body['time']
            # news_df.at[target_news_index,'clicked_times'] = total_click
        
        ### news_dic 만들기 끝. 여기부터는  user_dic 만들기
        add_click = news_id+ ','+body['publishtime']+ ','+body['time']
            
            ### 1) userId가 user_dic에 없는 경우: history, train, test 중 현재 파일의 폴더에 따라 해당 index에 데이터 저장
        if body['userId'] not in user_dic.keys():
            user_id = 'U'+str(user_cnt)
            user_dic[body['userId']] =[user_id]
            user_dic[body['userId']].append([]) # history
            user_dic[body['userId']].append([]) # train
            user_dic[body['userId']].append([]) # test
            user_dic[body['userId']][index].append([add_click]) # 각 폴더 데이터 body['userId']에 추가하는 부분; ex) index == 1이면 history
            user_cnt += 1

        user_dic[body['userId']][index].append([add_click])
            
            ### 2) train의 경우,해당 유저의 history 뉴스들을 가져와 history로 저장
        if data_type == "train":
            user_id = user_dic[body['userId']][0]
            history_list = list(set([j.split(',')[0] for i in user_dic[body['userId']][1] for j in i ]))
            write_line = user_id+'\t'+body['time'] + '\t' + ' '.join(history_list) + '\t'+news_id+'-1'
            behaviors_list.append(write_line)

            # target_user_index = int(user_dic[body['userId']][1:])-1
            # total_click = user_df.loc[target_user_index][data_type]+'\t'+add_click
            # user_df.at[target_user_index,data_type] = total_click
            
            ### 3) test의 경우 history, train의 뉴스들을 모두 가져와 user history로 저장
        elif data_type == "test":
            user_id = user_dic[body['userId']][0]
            """ 내 주석: history_list에는 history와 train 모두 사용 """
            history_list = list(set(
                                    [j.split(',')[0] for i in user_dic[body['userId']][1] for j in i ]
                                    + [j.split(',')[0] for i in user_dic[body['userId']][2] for j in i ]))
            write_line = user_id+'\t'+body['time'] + '\t' + ' '.join(history_list) + '\t'+news_id+'-1'
            behaviors_list.append(write_line)

    # 유효하지 않은 뉴스 수 계산
    else:
        filter1_cnt += 1




### Step 2. make_user_news_dic 실행하여 폴더마다 news/behaviors(raw).tsv 생성
### make_user_news_dic 정의 끝. 이제는 make_user_news_dic 실행하여 user.tsv 생성
########### user_behaviors filtering & news.tsv, user_behaviors.tsv 생성


index = 0

user_cnt = 1
news_cnt = 1
user_dic = {}
behavior_cnt = 0
total_cnt = 1

filter1_cnt = 1  ### 더미 데이터 제거
behaviors_user_filter_cnt = 1
time_error_cnt = 1

# start preprocessing
print()
print('Start preprocessing',end='    ')
print('preprocessing start_time:',time.strftime('%y-%m-%d %H:%M:%S'))
print()
print()

A = 1000000
# history, train, test 별 time_limit 설정
time_limit_list = [str_to_timestamp("2017-02-05 08:00:02"), str_to_timestamp("2017-02-12 08:00:02"), str_to_timestamp("2017-02-19 08:00:02")]
for data_type in dir_list:
    # news_df = pd.DataFrame(columns = ['news_id','publish_time','category','clicked_times','title','url'])
    news_dic = {}
    behaviors_list= []
    length = len(dir_list)
    index += 1
    print(f'[{data_type}] {index} / {length}',end='     ')
        # print('[missed data_type]', cnt)
    data_folder = os.path.join(NEWS_DIRECTORY, data_type)
    print(f'위치 :  {data_folder}')
    print()
    index2 = 1
    
    ### 각 raw data 파일 (.file)마다 for문 적용
    for news in os.listdir(data_folder):
        start2 = time.time()
        if news.endswith('.tsv') or news.endswith('.npy'):
            continue
        news_file = os.path.join(data_folder,news)
        # 진행 중인 파일 print
        print(f'{index2}/{len(os.listdir(data_folder))}     파일:{news_file}',end=' #')
        print('start_time:',time.strftime('%m-%d %H:%M:%S'),end=' #')


        ### 파일을 열어 각 줄 (body)마다 make_user_news_dic 실행
        index2 += 1
        file1 = open(news_file,'r')
        while True:
            total_cnt += 1
            line = file1.readline()
            if not line:
                break
            body = json.loads(line)
            
            ###################### time_limit 적용 부분            
            # body['time'] = parse_time(body['time'])
            # Time = str_to_timestamp(body['time'])
            # print("Time:", Time)
            # for i in range(100):
            #     print("yes")
            # sys.exit()
            # if index == 0: # 2017-02-05 08:00:02 이전까지만 적용
            #     if time_limit_list[index] < Time:
            #         continue
            # elif index == 1: # 2017-02-05 08:00:02 부터 2017-02-12 08:00:02 이전까지만 적용
            #     if time_limit_list[index-1] > Time or time_limit_list[index] < Time:
            #         continue
            # elif index == 2: # 2017-02-12 08:00:02 부터 2017-02-19 08:00:02 이전까지만 적용
            #     if time_limit_list[index-1] > Time or time_limit_list[index] < Time:
            #         continue
            
            
            # print(body.keys())

            make_user_news_dic(body)

        print('elapsed_time:',round(time.time()-start2,1))
        # a = len(user_dic)
        # if a >= A:
        #     print(f"A: {A}, a: {a} \n")
        #     exit()
            
        # print(f"A: {A}, a: {a} \n")
        # A = a
    
    
    ### 유저 필터링 전에 news/behaviors(raw).tsv 생성
    with open(os.path.join(data_folder,'news(raw).tsv'),'w',encoding='UTF-8') as wf:
        for key in news_dic.keys():
            wf.write(news_dic[key][0]+'\t'+news_dic[key][1]+'\t'+news_dic[key][2]+'\t'+','.join(news_dic[key][3])+'\t'+news_dic[key][4]+'\n')
    with open(os.path.join(data_folder,'behaviors(raw).tsv'),'w',encoding='UTF-8') as wf:
        for line in behaviors_list:
            wf.write(line+'\n')
    behavior_cnt += len(behaviors_list)
    print(f"{data_type} behavior_cnt",len(behaviors_list))
    
    
    
    
### user_dic 간단 체크
print()
print(len(user_dic))
print()
print()        
for key, value in itertools.islice(user_dic.items(), 10):
    print(f"{key}")
print()
print()
# print("exit")
# exit()
 
        
########## user filtering & user.tsv
"""Step 3. user(raw).tsv 생성"""


### 각 구간 별 클릭 없는 유저 제거 리스트
user_filter = []   
user_filter2_cnt = 1
user_filter3_cnt = 1
user_time_list = []
### user filtering 전 user(raw).tsv 생성
with open(os.path.join(NEWS_DIRECTORY,'user(raw).tsv'),'w',encoding='UTF-8') as wf2:
    for key in tqdm(user_dic.keys()):
        ### history, train, test의 클릭 수 합이 20보다 적으면 필터링
        if len(user_dic[key][1]) + len(user_dic[key][2]) + len(user_dic[key][3]) < 20: # 3가지 
            user_filter2_cnt += 1
            user_filter.append(key)

        # history, train, test 중 하나라도 클릭 수가 0이면 필터링
        elif (len(user_dic[key][1]) == 0) or (len(user_dic[key][2]) == 0) or (len(user_dic[key][3]) == 0):
            user_filter3_cnt += 1
            user_filter.append(key)
            
        ### 파일 생성
        else:
            writeline = user_dic[key][0]+'\t'
            for click in user_dic[key][1]:
                click = click[0]
                writeline += click+';'
                tmp = click.split(',')
                user_time_list.append(str_to_timestamp(tmp[2])-str_to_timestamp(tmp[1]))
            writeline.strip(';')
            writeline += '\t'
            for click in user_dic[key][2]:
                click = click[0]
                writeline += click+';'
                tmp = click.split(',')
                user_time_list.append(str_to_timestamp(tmp[2])-str_to_timestamp(tmp[1]))
            writeline.strip(';')
            writeline += '\t'
            for click in user_dic[key][3]:
                click = click[0]
                writeline += click+';'
                tmp = click.split(',')
                user_time_list.append(str_to_timestamp(tmp[2])-str_to_timestamp(tmp[1]))
            writeline.strip(';')
            writeline += '\n'
            wf2.write(writeline)
#     news_df.to_csv(os.path.join(data_folder,'news.tsv'),index=False)
# user_df.to_csv(os.path.join(NEWS_DIRECTORY,'user.tsv'),index=False)
print()
print(user_filter2_cnt)
print(user_filter3_cnt)
print()
print()
print('total_elapsed_time:',round(time.time()-start,1))
user_filter = set(user_filter)
user_filter_dic = {x:True for x in user_filter}  
        
        
        
        
        
"""Step 4. make_user_news_dic 재실행하여 user filtering 적용
   이때, 최종본인 news/behaviors.tsv 생성"""
        
        
""" make_user_news_dic 실행 끝.
    이제는 얻어진 user_filter_dic을 사용하여 make_user_news_dic 다시 생성
    뒤는 주석 user.tsv 부분만 다름"""      


########### user_behaviors filtering & news.tsv, user_behaviors.tsv
index = 0

user_cnt = 1
news_cnt = 1
user_dic = {}
behavior_cnt = 0
total_cnt = 1

filter1_cnt = 1  ### 더미 데이터 제거
behaviors_user_filter_cnt = 1
time_error_cnt = 1

# start preprocessing:
print()
print('Start preprocessing',end='    ')
print('preprocessing start_time:',time.strftime('%y-%m-%d %H:%M:%S'))
print()
print()

for data_type in dir_list:
    # news_df = pd.DataFrame(columns = ['news_id','publish_time','category','clicked_times','title','url'])
    news_dic = {}
    behaviors_list= []
    length = len(dir_list)
    index += 1
    print(f'[{data_type}] {index} / {length}',end='     ')
        # print('[missed data_type]', cnt)
    data_folder = os.path.join(NEWS_DIRECTORY, data_type)
    print(f'위치 :  {data_folder}')
    print()
    index2 = 1
    for news in os.listdir(data_folder):
        start2 = time.time()
        if news.endswith('.tsv') or news.endswith('.npy'):
            continue
        news_file = os.path.join(data_folder,news)
        print(f'{index2}/{len(os.listdir(data_folder))}     파일:{news_file}',end=' #')
        print('start_time:',time.strftime('%m-%d %H:%M:%S'),end=' #')


        index2 += 1
        file1 = open(news_file,'r')
        
        """first_execution False로 변경"""
        first_execution = False 
        while True:
            total_cnt += 1
            line = file1.readline()
            if not line:
                break
            body = json.loads(line)
            # print(body.keys())
            
            ###################### time_limit 적용 부분            
            # body['time'] = parse_time(body['time'])
            # Time = str_to_timestamp(body['time'])
            # if index == 0: # 2017-02-05 08:00:02 이전까지만 적용
            #     if time_limit_list[index] < Time:
            #         continue
            # elif index == 1: # 2017-02-05 08:00:02 부터 2017-02-12 08:00:02 이전까지만 적용
            #     if time_limit_list[index-1] > Time or time_limit_list[index] < Time:
            #         continue
            # elif index == 2: # 2017-02-12 08:00:02 부터 2017-02-19 08:00:02 이전까지만 적용
            #     if time_limit_list[index-1] > Time or time_limit_list[index] < Time:
            #         continue
            
            """여기서 first_execution=False이므로, user filtering 재수행"""
            make_user_news_dic(body)

        print('elapsed_time:',round(time.time()-start2,1))


    """수행된 유저 필터링을 바탕으로 최종 news/behaviors.tsv 생성"""
    with open(os.path.join(data_folder,'news.tsv'),'w',encoding='UTF-8') as wf:
        for key in news_dic.keys():
            wf.write(news_dic[key][0]+'\t'+news_dic[key][1]+'\t'+news_dic[key][2]+'\t'+','.join(news_dic[key][3])+'\t'+news_dic[key][4]+'\n')
    with open(os.path.join(data_folder,'behaviors.tsv'),'w',encoding='UTF-8') as wf:
        for line in behaviors_list:
            wf.write(line+'\n')
    behavior_cnt += len(behaviors_list)
    print(f"{data_type} behavior_cnt",len(behaviors_list))




"""Step 4. 최종본인 user.tsv 생성"""

"""수행된 유저 필터링을 바탕으로 최종 user.tsv 생성"""
user_filter = []   ### 각 구간 별 클릭 없는 유저 제거
user_filter2_cnt = 1
user_filter3_cnt = 1
user_time_list = []
with open(os.path.join(NEWS_DIRECTORY,'user.tsv'),'w',encoding='UTF-8') as wf2:
    for key in tqdm(user_dic.keys()):
        if len(user_dic[key][1]) + len(user_dic[key][2]) + len(user_dic[key][3]) < 20:
            user_filter2_cnt += 1
            user_filter.append(key)

        elif (len(user_dic[key][1]) == 0) or (len(user_dic[key][2]) == 0) or (len(user_dic[key][3]) == 0):
            user_filter3_cnt += 1
            user_filter.append(key)
        else:
            writeline = user_dic[key][0]+'\t'
            for click in user_dic[key][1]:
                click = click[0]
                writeline += click+';'
                tmp = click.split(',')
                user_time_list.append(str_to_timestamp(tmp[2])-str_to_timestamp(tmp[1]))
            writeline.strip(';')
            writeline += '\t'
            for click in user_dic[key][2]:
                click = click[0]
                writeline += click+';'
                tmp = click.split(',')
                user_time_list.append(str_to_timestamp(tmp[2])-str_to_timestamp(tmp[1]))
            writeline.strip(';')
            writeline += '\t'
            for click in user_dic[key][3]:
                click = click[0]
                writeline += click+';'
                tmp = click.split(',')
                user_time_list.append(str_to_timestamp(tmp[2])-str_to_timestamp(tmp[1]))
            writeline.strip(';')
            writeline += '\n'
            wf2.write(writeline)





print("news_cnt:",news_cnt)
print("user_cnt",user_cnt)
print("behavior_cnt",behavior_cnt)
print("total_cnt",total_cnt)
print("filter1_cnt",filter1_cnt)


Make_user_news_data    all_start_time: 24-11-13 20:20:36



Start preprocessing    preprocessing start_time: 24-11-13 20:20:36


[history] 1 / 3     위치 :  ./Adressa_4w\history

1/26     파일:./Adressa_4w\history\20170122 #start_time: 11-13 20:20:36 #Time: 1485039602.0
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes


SystemExit: 

c:\Users\Big Data Science LAB\anaconda3\envs\adressa_preprocess\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# total.news(raw).tsv 만들기

In [9]:
import time
from datetime import datetime

# 현재 Unix 타임스탬프를 연도, 날짜, 시간으로 변환
timestamp = time.time()
formatted_time = datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')
print(formatted_time)



data_type = 'Adressa_4w'
train_dir = f'C://Users//Big Data Science LAB//seongju//datasets//{data_type}/train'

behaviors = pd.read_table(
    os.path.join(train_dir,'behaviors.tsv'),
    header=None,
    names=['user', 'time', 'history', 'click'])

print(behaviors[:5])
print(behaviors[5:10])

2024-11-12 18:15:22
  user                 time                                   history    click
0   U1  2017-02-10 08:00:02                            N1 N1560 N1794  N1895-1
1   U2  2017-02-10 08:00:02  N2 N89 N68 N1100 N1103 N12 N1085 N109 N6  N1896-1
2   U3  2017-02-10 08:00:02                                   N3 N582  N1897-1
3   U4  2017-02-10 08:00:02        N1662 N1515 N1134 N4 N1725 N12 N16  N1898-1
4   U5  2017-02-10 08:00:02                                   N630 N5  N1899-1
  user                 time  \
5   U6  2017-02-10 08:00:02   
6   U7  2017-02-10 08:00:03   
7   U8  2017-02-10 08:00:03   
8   U9  2017-02-10 08:00:03   
9  U10  2017-02-10 08:00:03   

                                             history    click  
5                                           N1134 N6  N1900-1  
6                                        N4 N36 N417  N1898-1  
7                                             N22 N5  N1899-1  
8                                                 N7  N1901-1  

In [23]:
import os
import pandas as pd

# 폴더 경로 설정
NEWS_DIRECTORY = './Adressa_4w'
dir_list = ['history','train','test']
output_file = os.path.join(NEWS_DIRECTORY, 'total_news(raw).tsv')

# 빈 데이터프레임 생성
all_data = pd.DataFrame()

# 각 폴더에서 'news(raw).tsv' 파일 읽기
for folder in dir_list:
    file_path = os.path.join(NEWS_DIRECTORY, folder, 'news(raw).tsv')
    if os.path.exists(file_path):
        data = pd.read_csv(file_path, sep='\t', header=None)  # TSV 파일 불러오기
        all_data = pd.concat([all_data, data], ignore_index=True)  # 데이터프레임에 추가

# 컬럼 이름 지정 (기존 형식에 맞게 열의 이름을 부여)
all_data.columns = ['index', 'date', 'title', 'timestamps', 'category']

# 중복 제거: 'index' 컬럼을 기준으로 중복 제거 후, 첫 번째로 나타난 행 유지
all_data = all_data.drop_duplicates(subset='index')

# 'index' 열의 숫자 부분을 기준으로 정렬
all_data['index_number'] = all_data['index'].str.extract(r'(\d+)').astype(int)  # 숫자 부분 추출
all_data = all_data.sort_values(by='index_number').drop(columns=['index_number']).reset_index(drop=True)


# 결과 파일로 저장
all_data.to_csv(output_file, sep='\t', index=False, header=False)

print(f"파일이 생성되었습니다: {output_file}")


파일이 생성되었습니다: ./Adressa_4w\total_news(raw).tsv
